<a href="https://colab.research.google.com/github/Felipehonorato1/NLPcourse/blob/main/BERTClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow-text
!pip install -q tf-models-official

In [2]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization # Usaremos para criar o otimizador AdamW

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [6]:
 url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

 dataset = tf.keras.utils.get_file('aclImdb_v1.ta.gz', url, untar = True, cache_dir = ' .', cache_subdir ='')

84131840/84125825 [==============================] - 9s 0us/step


In [26]:
 dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
 print(dataset_dir)

 train_dir = os.path.join(dataset_dir, 'train')
 print(train_dir)

 test_dir = os.path.join(dataset_dir, 'test')
 print(test_dir)

/tmp/.keras/aclImdb
/tmp/.keras/aclImdb/train
/tmp/.keras/aclImdb/test


In [9]:
remove_dir = os.path.join(train_dir, 'unsup') # Removendo essa parte dos dados que não usaremos
shutil.rmtree(remove_dir)

In [14]:
# Métodos de otimização do algoritmo
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32

In [16]:
# Gerando nossos datasets de treino, validação e teste
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(train_dir, seed = 150, validation_split = 0.2, subset = 'training', batch_size=batch_size)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [18]:
val_ds = tf.keras.preprocessing.text_dataset_from_directory(train_dir,seed = 150, validation_split = 0.2 ,subset = 'validation', batch_size = batch_size )

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [27]:
test_ds = tf.keras.preprocessing.text_dataset_from_directory(test_dir, batch_size=batch_size)

Found 25000 files belonging to 2 classes.


In [19]:
class_names = raw_train_ds.class_names
print(class_names)

['neg', 'pos']


In [28]:
train_ds = raw_train_ds.cache().prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [30]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review: {text_batch.numpy()[i]}')
    print(f'Label: {class_names[label_batch.numpy()[i]]}')

Review: b"I've joined IMDb so people know what a great film this is! It's not often you come across a film that's moving and visually cinematic yet humble. You've read the plot so all I want to say is don't watch it because you want to see a clash of cultural religious identity babble ,because that's the typical misconception people read in to,instead just appreciate and realise it's about a father and son on a voyage growing to know each other through their struggles. Buy it and pass it on before film4 get round to it. This was one of the very few films to be nominated for a BAFTA being independent and foreign. The beauty of it is that it manages to appeal to anyone even if you never watch anything subtitled or just used to the Hollywood formula, just a great story that will keep you engaged. The only thing I wish is for it to be longer and see what happens"
Label: pos
Review: b'Before watching this film, I could already tell it was a complete copy of Saw (complete with the shack-like

In [44]:
# Usando o modelo BERT EN UNCASED 
def bert_embedding_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string) # Já que não temos tamanho definido para as strings passamos shape vazio
  preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

  encoder_inputs = preprocessor(text_input)

  encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=True)

  outputs = encoder(encoder_inputs)

  pooled_output = outputs['pooled_output']
  sequence_output = outputs = outputs['sequence_output']
  model = tf.keras.Model(text_input, pooled_outputs)

  return model



In [39]:
embedding_model = bert_embedding_model()

In [40]:
sentences = tf.constant(['This is a test'])
processed_sentences = embedding_model(sentences)

In [70]:
# BERT UNCASED MODEL
def classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name = 'text input') # Já que não temos tamanho definido para as strings passamos shape vazio
  preprocessed = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',name = 'Preprocessing_layer')(text_input)

  encoder_outputs = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=True, name = 'Encoding')(preprocessed)

  pooled_output = encoder_outputs['pooled_output']
  
  net = tf.keras.layers.Dropout(0.1, name = 'Dropout_layer')(pooled_output)
  net = tf.keras.layers.Dense(16, activation ='relu', name = 'Dense_net')(net)
  net = tf.keras.layers.Dense(1,activation = 'sigmoid', name = 'Classifier')(net)

  model = tf.keras.Model(text_input, net)

 

  init_lr = 3e-5
  epochs = 5
  steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
  num_train_steps = steps_per_epoch * epochs
  num_warmup_steps = int(0.1*num_train_steps)
  optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  metrics = tf.metrics.BinaryAccuracy()
  model.compile(optimizer = optimizer, loss = loss, metrics = metrics)
  return model

  

In [71]:
classifier = classifier_model()

In [ ]:
history = classifier.fit(x = train_ds, validation_data= val_ds, epochs = 5)

Epoch 1/5
625/625 [==============================] - 1178s 2s/step - loss: 0.5118 - binary_accuracy: 0.7138 - val_loss: 0.2873 - val_binary_accuracy: 0.8782
Epoch 2/5
625/625 [==============================] - ETA: 0s - loss: 0.2678 - binary_accuracy: 0.8915